In [59]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Performance

## How programmers affects performance?

In [ ]:
compare([do_render_code("./programmer/madd_A.c", lang="c++", show="main"),
do_render_code("./programmer/madd_B.c", lang="c++", show="main")])

In [67]:
! cd programmer; make clean; make
! echo "running version A"; perf stat taskset -c 8 ./programmer/madd_A
! echo "running version B"; perf stat taskset -c 8 ./programmer/madd_B

rm -f madd_A madd_B madd_change_freq madd_A_unroll *_O3 *~
#hotspot hotspot_omp
cc -O0 -DHAVE_LINUX_PERF_EVENT_H -g  madd_A.c -o madd_A
cc -O0 -DHAVE_LINUX_PERF_EVENT_H -g  madd_B.c -o madd_B
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3  madd_A.c -o madd_A_O3
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3  madd_B.c -o madd_B_O3
cc -O3 -DHAVE_LINUX_PERF_EVENT_H -g -DA madd_change_freq.c perfstats.o -o madd_change_freq
madd_change_freq.c: In function ‘main’:
madd_change_freq.c:18:3: warning: implicit declaration of function ‘change_cpufrequnecy’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
   18 |   change_cpufrequnecy(freq);
      |   ^~~~~~~~~~~~~~~~~~~
madd_change_freq.c:49:3: warning: implicit declaration of function ‘restore_cpufrequnecy’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
   49 |   restore_cpufrequnecy();
   

In [ ]:
! lscpu

<div class="alert alert-block alert-info">
    
### Question:

What does the programmer affect?
    
</div>

In [68]:
compare([do_render_code("../introduction/arraySort.cpp",lang="c++", show="main"),do_render_code("../introduction/arraySort.cpp",lang="c++", show="calculate_sum")])

In [71]:
! cd ../introduction/; make clean; make; echo "not sorting"; perf stat  taskset -c 8 ./arraySort 262144 1000 0; echo "sorting"; perf stat   taskset -c 8 ./arraySort 262144 1000 1

rm -f madd arraySort bitonic quicksort mm quicksort_cpu
gcc madd.c -o madd 
gcc calculate_sum.c -c 
g++ -O0 arraySort.cpp calculate_sum.o -o arraySort
/usr/local/cuda/bin/nvcc -O3 bitonic.cu -o bitonic
/usr/local/cuda/bin/nvcc -O3 quicksort.cu -o quicksort
gcc -O3 mm.c perfstats.o -o mm 
/usr/local/cuda/bin/nvcc -O3 quicksort_cpu.cpp -o quicksort_cpu
not sorting
sum = 127161197

 Performance counter stats for 'taskset -c 8 ./arraySort 262144 1000 0':

            611.56 msec task-clock                       #    0.999 CPUs utilized             
                 6      context-switches                 #    9.811 /sec                      
                 1      cpu-migrations                   #    1.635 /sec                      
               453      page-faults                      #  740.733 /sec                      
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
     3,137,208,199      cpu_core/cycles/                 #  

<div class="alert alert-block alert-info">
    
### Question:

What does the programmer affect?
    
</div>

In [ ]:
render_code("./programmer/perfstats.c", lang="c++", show="change_cpufrequnecy")

In [ ]:
!cpupower frequency-info -n

In [ ]:
! cat /proc/cpuinfo|grep MHz

In [ ]:
! cs203 job run "perf stat ./programmer/madd_change_freq 8192 800"
! cs203 job run "perf stat ./programmer/madd_change_freq 8192 1700"
! cs203 job run "perf stat ./programmer/madd_change_freq 8192 3300"

## How programming languages affects performance?

### Python
    

In [44]:
render_code("./programming_languages/hello.py")

// ./programming_languages/hello.py:1-1 (1 lines)
print("Hello, World!")

In [81]:
!perf stat  taskset -c 8 python ./programming_languages/hello.py

Hello, World!

 Performance counter stats for 'taskset -c 8 python ./programming_languages/hello.py':

             12.80 msec task-clock                       #    0.937 CPUs utilized             
                 4      context-switches                 #  312.395 /sec                      
                 2      cpu-migrations                   #  156.197 /sec                      
               970      page-faults                      #   75.756 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
        32,114,903      cpu_core/cycles/                 #    2.508 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
        50,522,666      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
        10,669,

### Perl

In [46]:
render_code("./programming_languages/hello.pl")

// ./programming_languages/hello.pl:1-5 (5 lines)
#!/usr/bin/perl
use strict;
use warnings;

print "Hello, World!\n";

In [79]:
! perf stat  taskset -c 8 ./programming_languages/hello.pl

Hello, World!

 Performance counter stats for 'taskset -c 8 ./programming_languages/hello.pl':

              9.12 msec task-clock                       #    0.678 CPUs utilized             
                 6      context-switches                 #  657.575 /sec                      
                 1      cpu-migrations                   #  109.596 /sec                      
               376      page-faults                      #   41.208 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
         9,904,802      cpu_core/cycles/                 #    1.086 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
        15,944,890      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
         3,237,477    

### Java

In [48]:
render_code("./programming_languages/HelloWorld.java")

// ./programming_languages/HelloWorld.java:1-8 (8 lines)
public class HelloWorld {

    public static void main(String[] args) {
        // Prints "Hello, World" to the terminal window.
        System.out.println("Hello, World");
    }

}

In [78]:
! cd ./programming_languages/; javac HelloWorld.java
! cd ./programming_languages/; perf stat  taskset -c 8 java HelloWorld

Hello, World

 Performance counter stats for 'taskset -c 8 java HelloWorld':

             28.50 msec task-clock                       #    0.830 CPUs utilized             
                92      context-switches                 #    3.228 K/sec                     
                 1      cpu-migrations                   #   35.090 /sec                      
             5,619      page-faults                      #  197.171 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
        80,545,680      cpu_core/cycles/                 #    2.826 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
       104,293,881      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
        19,138,737      cpu_core/branche

### Go

In [50]:
render_code("./programming_languages/hello-world.go")

// ./programming_languages/hello-world.go:1-5 (5 lines)
package main
import "fmt"
func main() {
    fmt.Println("hello world")
}

In [72]:
! perf stat  taskset -c 8 /usr/local/go/bin/go run ./programming_languages/hello-world.go

hello world

 Performance counter stats for 'taskset -c 8 /usr/local/go/bin/go run ./programming_languages/hello-world.go':

             94.87 msec task-clock                       #    0.494 CPUs utilized             
             4,262      context-switches                 #   44.923 K/sec                     
                 1      cpu-migrations                   #   10.540 /sec                      
            10,646      page-faults                      #  112.213 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
       297,224,309      cpu_core/cycles/                 #    3.133 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
       596,789,308      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (

In [73]:
!/usr/local/go/bin/go build ./programming_languages/hello-world.go
! perf stat  taskset -c 8 ./hello-world

hello world

 Performance counter stats for 'taskset -c 8 ./hello-world':

              5.05 msec task-clock                       #    0.271 CPUs utilized             
                59      context-switches                 #   11.692 K/sec                     
                 0      cpu-migrations                   #    0.000 /sec                      
               239      page-faults                      #   47.361 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
         5,043,509      cpu_core/cycles/                 #    0.999 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
         7,180,929      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
         1,267,977      cpu_core/branches/ 

### C++

In [53]:
render_code("./programming_languages/hello.cpp")

// ./programming_languages/hello.cpp:1-8 (8 lines)
#include <iostream>
using namespace std;

int main() 
{
    cout << "Hello, World!";
    return 0;
}

In [75]:
! g++ -O4 ./programming_languages/hello.cpp -o ./programming_languages/hello_cpp
! perf stat  taskset -c 8 ./programming_languages/hello_cpp

Hello, World!
 Performance counter stats for 'taskset -c 8 ./programming_languages/hello_cpp':

              3.61 msec task-clock                       #    0.653 CPUs utilized             
                 3      context-switches                 #  830.792 /sec                      
                 1      cpu-migrations                   #  276.931 /sec                      
               193      page-faults                      #   53.448 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
         3,880,983      cpu_core/cycles/                 #    1.075 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
         5,020,736      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
           909,006    

### C

In [ ]:
render_code("./programming_languages/hello.c")

In [82]:
! gcc -O4 ./programming_languages/hello.c -o ./programming_languages/hello_c
! perf stat  taskset -c 8 ./programming_languages/hello_c

Hello, world!

 Performance counter stats for 'taskset -c 8 ./programming_languages/hello_c':

              2.18 msec task-clock                       #    0.546 CPUs utilized             
                 3      context-switches                 #    1.377 K/sec                     
                 1      cpu-migrations                   #  458.884 /sec                      
               126      page-faults                      #   57.819 K/sec                     
     <not counted>      cpu_atom/cycles/                                                        (0.00%)
         2,306,204      cpu_core/cycles/                 #    1.058 GHz                       
     <not counted>      cpu_atom/instructions/                                                  (0.00%)
         2,400,745      cpu_core/instructions/                                                
     <not counted>      cpu_atom/branches/                                                      (0.00%)
           442,983     

## Compiler

In [26]:
! cd programmer; make clean; make madd_A
! echo "running version A (Unoptimized)"; time ./programmer/madd_A

rm -f madd_A madd_B madd_change_freq madd_A_unroll *_O3 *~
#hotspot hotspot_omp
cc -O0 -DHAVE_LINUX_PERF_EVENT_H -g  madd_A.c -o madd_A
running version A (Unoptimized)
161

real	0m1.676s
user	0m1.524s
sys	0m0.151s


In [27]:
! cd programmer; make clean; make madd_B
! echo "running version B (Unoptimized)"; time ./programmer/madd_B

rm -f madd_A madd_B madd_change_freq madd_A_unroll *_O3 *~
#hotspot hotspot_omp
cc -O0 -DHAVE_LINUX_PERF_EVENT_H -g  madd_B.c -o madd_B
running version B (Unoptimized)
161

real	0m2.574s
user	0m2.436s
sys	0m0.136s


In [28]:
! cd programmer; make madd_A_O3
! echo "running version A (Optimized)"; time ./programmer/madd_A_O3

cc -DHAVE_LINUX_PERF_EVENT_H -g -O3  madd_A.c -o madd_A_O3
running version A (Optimized)
161

real	0m1.541s
user	0m1.383s
sys	0m0.157s


In [29]:
! cd programmer; make madd_B_O3
! echo "running version B (Optimized)"; time ./programmer/madd_B_O3

cc -DHAVE_LINUX_PERF_EVENT_H -g -O3  madd_B.c -o madd_B_O3
running version B (Optimized)
161

real	0m2.189s
user	0m2.013s
sys	0m0.174s


<div class="alert alert-block alert-info">
    
### Question:

What's the limitation of compiler?
    
</div>

In [37]:
compare([do_render_code("compiler/arithmetics.c", show="a_mul_43"), do_render_code("compiler/arithmetics.c", show="a_mul_43_no_opt")])

In [62]:
render_code("compiler/arithmetics.c", show="main")
! cd compiler; gcc -O3 -DHAVE_LINUX_PERF_EVENT_H perfstats.c arithmetics.c -o arithmetics
! echo "IC,Cycles,CPI,CycleTime,ET,DL1_Miss_Rate,DL1_MISSES,DL1_ACCESSES" > ./compiler/stats.csv
! ./compiler/arithmetics 2>> compiler/stats.csv
display_df_mono(render_csv("compiler/stats.csv",columns=["IC", "CPI", "CycleTime", "ET"]))

// compiler/arithmetics.c:32-73 (42 lines)
int main(int argc, void **argv)
{
    clock_t start, stop;
    unsigned N = 16777216;
    int64_t *data, *data_2, i, r;
    if(argc > 1)
        N = atoi(argv[1]);
    data = (int64_t *)malloc(sizeof(int64_t) *N);
    data_2 = (int64_t *)malloc(sizeof(int64_t) *N);
    memcpy(data, data_2, sizeof(int64_t) *N);
    r = rand();
    start = clock();
    perfstats_init();
    perfstats_enable();
    for(i=0;i<N;i++)
        data[i] = a_mul_43_no_opt(data[i]);
    perfstats_disable();
    perfstats_print();
    perfstats_deinit();
    stop = clock();
    printf("No optimization\n");
    print_elapsed(start, stop);
    start = clock();
    perfstats_init();
    perfstats_enable();
    for(i=0;i<N;i++)
        data_2[i] = a_mul_43(data_2[i]);
    perfstats_disable();
    perfstats_print();
    perfstats_deinit();
    stop = clock();
    printf("Optimized\n");
    print_elapsed(start, stop);
    r = r%N;
    printf("%ld\n",a_mul_43(data[r]));
    printf("%ld\n",a_div_43(data[r]));
    printf("%ld\n",a_mod_43(data[r]));
    printf("%ld\n",a_mul_43(data_2[r]));
    printf("%ld\n",a_div_43(data_2[r]));
    printf("%ld\n",a_mod_43(data_2[r]));    
    return 0;
}

No optimization
Elapsed time: 0.014s
Optimized
Elapsed time: 0.034s
0
0
0
0
0
0


,index,IC,CPI,CycleTime,ET
0,0,335710257,0.216007,0.193682,0.014045
1,1,290390318,0.597409,0.193548,0.033577
